## Código para testar o modelo de YOLO

---

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
from paddleocr import PaddleOCR
import os
import re
import requests

### Identificação de placas de carros:

In [ ]:
def classify_image(image_path):    
    model = YOLO('last.pt')  
    results = model(image_path)
    
    # Display results
    for result in results:
        img_rgb = cv2.cvtColor(result.plot(show=False), cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 10))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.show()

def classify_and_crop(image_path):
    # Load model
    model = YOLO('best.pt')
    
    # Run inference
    results = model(image_path)
    
    # Read the original image
    original_image = cv2.imread(image_path)
    imgnames = []


    # Iterate over results
    for i, result in enumerate(results):
        boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes in (x1, y1, x2, y2) format

        for j, (x1, y1, x2, y2) in enumerate(boxes):
            # Convert coordinates to integers
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Crop the image
            cropped = original_image[y1:y2, x1:x2]

            # Convert BGR to RGB for displaying
            cropped_rgb = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
            # Save the cropped image
            output_filename = f'cropped_{i}_{j}.png'
            imgnames.append(output_filename)
            cv2.imwrite(output_filename, cropped)

            # Show the cropped region
            plt.figure(figsize=(4, 4))
            plt.imshow(cropped_rgb)
            plt.axis('off')
            plt.title(f'Cropped #{i}-{j}')
            plt.show()
    return imgnames

### Lendo o texto das placas

In [ ]:
def change_char_in_position(word, position):
    # Change the character at the specified position to a letter
    if position < len(word):
        if word[position].isdigit():
            digit = word[position]
            # Se for 8 troca por B, se for 1 troca por I
            if digit == '8':
                word = word[:position] + 'B' + word[position+1:]
            elif digit == '1':
                word = word[:position] + 'I' + word[position+1:]
    return word

ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log=False)

def extract_plate_from_image(image_path):
    # Load model
    result = ocr.ocr(image_path, cls=True)
    
    if not result or result[0] == None:
        print("No text detected.")
        return None    
    is_new_plate = False
    
    detected_words = []
    for line in result:
        for word_info in line:
            if word_info[1][0].lower() == "brasil":
                is_new_plate = True
            else:
                detected_words.append((word_info[1][0], word_info[1][1]))
                
                
    print(f"É nova placa? {is_new_plate}")    
    for word_tuple in detected_words:
        word = word_tuple[0].replace("-", "")
        confidence = word_tuple[1]                
            
        # Deve ter o formato "XXX9999" ou "XXX9X99"
        
        if (re.fullmatch(r'[A-Z]{3}\d{4}', word) and not is_new_plate) or re.fullmatch(r'[A-Z]{3}\d[A-Z]\d{2}', word):
            return word
        
        else:
            # Try to fix plate text: characters 0,1,2 must be letters.
            if word[0].isdigit():
                word = change_char_in_position(word, 0)
            if word[1].isdigit():
                word = change_char_in_position(word, 1)
            if word[2].isdigit():
                word = change_char_in_position(word, 2)
            if is_new_plate and word[4].isdigit():
                word = change_char_in_position(word, 4)
                            
            if re.match(r'^[A-Z]{3}\d{4}$', word) or re.match(r'^[A-Z]{3}\d{1}[A-Z]\d{2}$', word):
                return word
            
    return None

### Testando com imagens reais

In [ ]:
image_path = 'imgs/5.jpg'
imgs = classify_and_crop(image_path)

In [ ]:
plates = []
for img in imgs:
    print(f"Processing image: {img}")
    plate = extract_plate_from_image(img)
    print(f"Plate detected: {plate}")
    if plate:
        plates.append(plate)
    print('-'*20)


----

In [ ]:
image_path = '../compvis-car/Brazil/files/domain1/7471000.jpg'
imgs = classify_and_crop(image_path)

In [ ]:
plates = []
for img in imgs:
    print(f"Processing image: {img}")
    plate = extract_plate_from_image(img)
    print(f"Plate detected: {plate}")
    if plate:
        plates.append(plate)
    print('-'*20)


In [ ]:
token = os.getenv('API_TOKEN', 'não-encontrado')
token

In [ ]:
api_url = 'https://wdapi2.com.br/consulta/{placa}/{token}'

placa = plates[1]
token = os.getenv('API_TOKEN', 'default_token_value')
response = requests.get(api_url.format(placa=placa, token=token))
if response.status_code == 200:
    data = response.json()
    print(f"API Response: {data}")

In [ ]:
def get_infos(data):
    info = {}
    marca = data['marca']
    modelo = data['modelo']
    ano = data['ano']
    cor = data['cor']

    info = {
        'marca': marca,
        'modelo': modelo,
        'ano': ano,
        'cor': cor
    }

    extra = data['extra']

    campos_interesse = ['municipio', 'nacionalidade', 'sub_segmento']
    for chave in campos_interesse:
        if chave in extra:
            info[chave] = extra[chave]
        else:
            info[chave] = None
            
    fipe = data['fipe']['dados']
    sum = 0
    i = 0
    for dados in fipe:
        valor = dados['texto_valor']
        valor = valor.replace("R$", "")
        valor = valor.replace(".", "")
        valor = valor.replace(",", ".")
        valor = float(valor)
        sum += valor
        i += 1
        
    media = sum / i
    info['fipe'] = media
    
    return info

info = get_infos(data)
print(info)
            